In [50]:
import numpy as np

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import os
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

sns.set(style='darkgrid', palette='viridis', context='talk')


# Import simulation output

Before running this, I ran a sim in ecoli/experiments/new_environments.py with -n 0 to generate a simulation output. This is necessary to replicate the simulation environment, where we e.g. have dynamic demand for amino acids and nucleotides, and varying cell volumes.

In [51]:
flux_save = pd.DataFrame()

condition_df = pd.read_excel("ecoli/experiments/new_environments_test/done.xlsx")
condition_df["Add"] = condition_df["Add"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Remove"] = condition_df["Remove"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Demand"] = condition_df["Demand"].apply(lambda x: set(str(x).split(", ") if x == x else set()))

for i in range(len(condition_df)):
    time = '50'
    date = '2023-07-21'
    experiment = 'fba_new_environments'
    env_name = f'{condition_df.loc[i, "Environment"]}'
    entry = f'{experiment}_{env_name}_{time}_{date}'
    folder = f'out/fba_new_env/{entry}/'
    try:
        output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
        # output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
        output = output['agents']['0']
        fba = output['listeners']['fba_results']
        mass = output['listeners']['mass']
        bulk = pd.DataFrame(output['bulk'])
        # this is the model itself - we'll pull e.g. stoichiometry from here.
        f = open(folder + 'agent_steps.pkl', 'rb')
        agent = dill.load(f)
        f.close()

        metabolism = agent['ecoli-metabolism-redux']
        stoichiometry = metabolism.stoichiometry

        # sim_exchanges = pd.DataFrame(fba["estimated_exchange_dmdt"])
        # sim_exchanges = pd.DataFrame(sim_exchanges.loc[24, :].abs().sort_values(ascending=False))
        # if os.path.exists(f'out/new_env_analysis/{env_name}'):
        #     pass
        # else:
        #     os.mkdir(f'out/new_env_analysis/{env_name}')
        # sim_exchanges.to_csv(rf'out/new_env_analysis/{env_name}/sim_exchanges.csv')

        sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
        sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

        if i == 0:
            flux_save = sim_fluxes
        else:
            new_col = sim_fluxes.iloc[:,0].to_frame()
            flux_save[env_name] = new_col
            # flux_save[0, np.shape(flux_save)[1] - 1] = env_name
        # sim_fluxes.to_csv(rf'out/new_env_analysis/{env_name}/sim_fluxes.csv')

        # sim_catalysts = pd.DataFrame(fba["reaction_catalyst_counts"], columns=metabolism.reaction_names)
        # sim_catalysts = pd.DataFrame(sim_catalysts.loc[4, :].abs().sort_values(ascending=False))
        # sim_catalysts.to_csv(rf'out/new_env_analysis/{env_name}/sim_catalysts.csv')

    except FileNotFoundError:
        pass

In [52]:
df = pd.DataFrame(flux_save)
flux_save.to_csv(rf'out/new_env_analysis/overall_sim_fluxes.csv')

                                                            24  \
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.  20133261.0   
RXN0-5256-CPD-9728/PROTON/HYDROGEN-MOLECULE//RE...  20101063.0   
TRANS-RXN-106                                       20033460.0   
TRANS-RXN-145                                       10477226.0   
ATPSYN-RXN (reverse)                                 9599983.0   
...                                                        ...   
RXN-21538                                                  0.0   
RXN-21533-CPD-23688/ATP//CPD-23428/ADP/PROTON.36.          0.0   
RXN-21533-CPD-23431/ATP//CPD-23690/ADP/PROTON.36.          0.0   
RXN-21533-CPD-23431/ATP//CPD-23428/ADP/PROTON.36.          0.0   
RXN0-7200-NMNH//NMNH.11. (reverse)                         0.0   

                                                    Ribose_C-source  \
R601-RXN-FUM/REDUCED-MENAQUINONE//SUC/CPD-9728.38.              0.0   
RXN0-5256-CPD-9728/PROTON/HYDROGEN-MOLECULE//RE...              0

## Analysis on spreadsheet with all fluxes for all environments

In [ ]:
overall_flux = pd.read_excel("ecoli/out/new_env_analysis/overall_sim_fluxes.xlsx")